# Weighted Jaccard Similarity

In this notebook, we will use cuGraph to compute the Weighted Jaccard Similarity metric on our training dataset. 


Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 04/30/2019

RAPIDS Versions: 0.7.0  


Test Hardware
* GP100 32G, CUDA 9,2


## Introduction
Weighted Jaccard is similar to the Jaccard Similarity but takes into account vertex weights placed.  


given:
The neighbors of a vertex, v, is defined as the set, U, of vertices connected by way of an edge to vertex v, or N(v) = {U} where v ∈V and ∀ u∈U ∃ edge(v,u)∈E.
and
wt(i) is the weight on vertex i
   
we can now define weight summing function as<br>
$WT(U) = \sum_{v \in U} {wt(v)}$

$WtJaccard(i, j) = \frac{WT(N(i) \cap N(j))}{WT(N(i) \cup N(j))}$


To compute the weighted Jaccard similarity between each pair of vertices connected by an edge in cuGraph use:<br>

**jaccard_w(input_graph, vect_weights_ptr)**

Input
* input_graph: A cugraph.Graph object
* vect_weights_ptr: An array of vertex weights

Returns: 
* __df__: cudf.DataFrame with three columns:
    * df['source']: The source vertex id.
    * df['destination']: The destination vertex id.
    * df['jaccard_coeff']: The weighted jaccard coefficient computed between the source and destination vertex.
        
<br><br>   


__Note:__ For this example we will be using PageRank as the edge weights.  Please review the PageRank notebook if you have any questions about running PageRank


    
## cuGraph 0.7 Notice 
cuGraph version 0.7 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations 

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

## Read the data using cuDF

In [2]:
# Test file   
datafile='./data/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [ ]:
# Since the test data has the first vertex starting at index "1" and cuGraph is zero based, 
# adjust the indexies
gdf['src_id'] = gdf['src'] -1
gdf['dst_id'] = gdf['dst'] -1

In [3]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_id"], gdf["dst_id"])

### Compute PageRank to use as the vertex weights

In [4]:
# Call Pagerank on the graph to get weights to use:
pr_df = cugraph.pagerank(G)

In [5]:
pr_df.head().to_pandas()

,vertex,pagerank
0,0,0.004392
1,1,0.096573
2,2,0.052645
3,3,0.056828
4,4,0.035703


### Now compute the Weighted Jaccard 

In [6]:
# Call weighted Jaccard using the Pagerank scores as weights:
df = cugraph.jaccard_w(G, pr_df['pagerank'])

Find the most similar pair of vertices

In [7]:
bestEdge = 0
for i in range(len(df)):
    if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
        bestEdge = i
        
print("Vertices " + str(df['source'][bestEdge]) + 
      " and " + str(df['destination'][bestEdge] ) + 
      " are most similar with score: " + str(df['jaccard_coeff'][bestEdge]))

Vertices 4 and 8 are most similar with score: 0.6644074


## It is that easy with cuGraph!

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___